## Rest-tehtävät

Ensimmäisessä tehtäväosiossa rajapintaa pääkaupunkiseudun palveluista. Tämä rajapinta on myös käytössä esimerkeissä.

Tietoa palvelusta löytyy: https://www.hel.fi/palvelukarttaws/restpages/palvelurekisteri.html

---

Toisessa tehtäväosiossa on käytössä Digitrafficin rest-rajapinta, johon on tarkoitus soveltaa osaamista. 

Tietoa palvelusta löytyy: https://www.digitraffic.fi/rautatieliikenne/#dataa-tukevat-rajapinnat--ty%C3%B6kalut

### Pääkaupunkiseudun palveluiden rajapinnan tehtävät

Url-osoite rajapinnalle: http://www.hel.fi/palvelukarttaws/rest/vpalvelurekisteri

#### Tehtävä 1. Lisää käytettävät kirjastot ja muodosta rajapinnan url-osoite päätteellä `/description/`

In [1]:
# tuodaan kirjasto, jolla voidaan tehdä http-pyyntöjä ja tuodaan json
import requests
import json
import pprint
import pandas as pd

In [2]:
# muodostetaan yhteys haluttuun rest-rajapintaan
url = "http://www.hel.fi/palvelukarttaws/rest/vpalvelurekisteri/description/"

resp = requests.get(url=url)

#### Tehtävä 2. Tee get-pyyntö osoitteeseen ja varmista, että haku onnistuu ja vastauskoodina on 200

In [3]:
# varmistetaan, että haku onnistuu vastauskoodista
print("Vastauskoodi:", resp.status_code)

Vastauskoodi: 200


#### Tehtävä 3. Tallenna vastaus muuttujaan listana, joiden elementit ovat json-muodossa

In [4]:
# luetaan vastaus ja tehdään siitä muuttuja
data = resp.json()
print(type(data))

<class 'list'>


#### Tehtävä 4. Valitse list-objektista viimeisin json-tietue, jonka `service_id`-avaimen arvo on 10473 (10473 arvoja on kolme) ja sijoita kyseisen json-tietueen `id`-avaimen arvo muuttujaan.

In [6]:
# tehdään kysely rajapintaan halutulla id:llä
id_resp = "http://www.hel.fi/palvelukarttaws/rest/vpalvelurekisteri/description/4892"

#### Tehtävä 5. Tee uusi get-pyyntö samaan osoitteeseen kuin aiemmin (myös päätepiste pysyy samana), mutta ota edellisessä kohdassa valittu muuttuja (muuttuja joka sisältää `id`:n arvon) huomioon pyynnössä

In [10]:
# tehdään uusi get-pyyntö, että saadaan data json-muodossa
resp2 = requests.get(url=id_resp)
resp2

<Response [200]>

#### Tehtävä 6. Tulosta vastauksesta `title`-avaimen arvo

In [12]:
# tulostetaan json-datasta halutun id:n arvo
data2 = resp2.json()
print(data2['title'])

Itämerihaaste


---

### Digitrafficin rajapinnan tehtävät

#### Tehtävä 7. Hae Digitrafficin rajapinnasta palvelun ohjeiden mukaan tiedot junalle päivältä 06.06.2018, jonka numero on 1

In [14]:
# hakukysely tietylle junalle tietyltä päivältä ja vastauskoodin tulostus konsoliin
train_url = 'https://rata.digitraffic.fi/api/v1/trains/2018-06-06/1'
train_r = requests.get(url=train_url)

# tulostetaan vastauskoodi konsoliin
print(train_r)

<Response [200]>


In [15]:
# valmistellaan data python-muotoon ja käsiteltäväksi seuraavaan tehtävään
train_info = json.loads(train_r.text)[0]

#### Tehtävä 8. Muodosta edellisen tehtävän get-pyynnön tuloksesta DataFrame (koko Junan 1 päivän 06.06.2018 datasta), jos yksikin `scheduledTime`-avaimen arvo on ennen aikaleimaa '2018-06-06T06:00:00.000Z'.

In [1]:
# lista johon tallennetaan junien aikataulutiedot str-muodossa
timetable = []


# tehdään funktio, jonka avulla haetaan datasta listaan kaikki junan aikataulutiedot
def get_timetable():
    station_access = train_info['timeTableRows']
    for stations in station_access:
        timetable_access = stations['scheduledTime']
        timetable.append(timetable_access)
get_timetable()


# muutetaan junan aikataulu str -> vertailtavaksi aikaformaatiksi pandasin avulla
timetable_series = pd.Series(timetable)
timetable_series = pd.to_datetime(timetable_series)


# tehdään tarkistus löytyykö junan lähtöaikoja ennen annettua aikamäärettä
timecheck = timetable_series <= pd.to_datetime('2018-06-06T06:00:00.000Z')


# jos löytyy aikoja ennen aikamäärettä muodostetaan DataFrame junan tiedoista
if pd.Series.any(timecheck) == True:
    print("Löytyi aika!")
    df = pd.read_json(train_r.text)
else:
    print("Ei löytynyt yhtään lähtöaikaa ennen annettua aikamäärettä")

NameError: name 'train_info' is not defined

#### Tehtävä 9. Tulosta ainoa rivi generoidusta DataFrame-objektista, jos sellainen muodostetaan.

In [17]:
# tulostetaan dataframe aiemman tehtävän tuottaman tuloksen perusteella

df

,trainNumber,departureDate,operatorUICCode,operatorShortCode,trainType,trainCategory,commuterLineID,runningCurrently,cancelled,version,timetableType,timetableAcceptanceDate,timeTableRows
0,1,2018-06-06,10,vr,IC,Long-distance,,False,False,270528660210,REGULAR,2018-03-05T10:02:10.000Z,"[{'stationShortCode': 'HKI', 'stationUICCode':..."
